# Day 11: Dumbo Octopus
https://adventofcode.com/2021/day/11

### Part 1
Count the Flashes from the energy Octopuses after 100 rounds.

In [1]:
import os
from dataclasses import dataclass, field
from typing import Tuple


In [2]:
# input data.
def test_input_location(
    file_loc: str = 'test_input.txt', 
    data_directory: str  = 'data/day_11'
) -> str:
    return os.path.join(data_directory, file_loc)

def input_location(
    file_loc: str = 'input.txt', 
    data_directory: str  = 'data/day_11'
) -> str:
    return os.path.join(data_directory, file_loc)


In [3]:
@dataclass(order=True, eq=True) 
class Octopus:
    row: int = field(default=0, compare=True)
    col: int = field(default=0, compare=True)
    engergy_level: int = field(default=0, compare=False)
    flashed_this_round: bool = field(default=False, compare=False, repr=False)

    def increment_energy(self) -> bool:
        """
        Increments energy level by 1.  If energy level reaches >9, it will flash (and return True).
        If it already flashed this round, it will not flash again, and its energy will be zero.
        """
        new_flash = False 

        if not self.flashed_this_round and self.engergy_level < 9:
            self.engergy_level = self.engergy_level + 1
        elif not self.flashed_this_round and self.engergy_level >= 9:
            self.flashed_this_round = True 
            self.engergy_level = 0
            new_flash = True
        
        return new_flash


class Cave:
    grid: list[list[Octopus]]

    def __init__(self, input_file:str):
        self.grid = list()

        if os.path.exists(input_file):
            with open(input_file) as f:    
                for row, line in enumerate(f):
                    if line.rstrip():
                        octopuses: list[Octopus] = list()
                        for col, initial_energy in enumerate(line.strip()):
                            octopuses.append(Octopus(row, col, int(initial_energy)))
                        self.grid.append(octopuses)

    def __repr__(self) -> str:
        grid = ""
        for ridx, row in enumerate(self.grid):
            grid = grid + "".join([str(col.engergy_level) for cidx, col in enumerate(row)]) + "\n"
        return grid

    def increment_energy_level_at(self, row:int, col:int) -> bool:
        """
        Increments the engery of the Octopus at this grid space (row, col).
        Returns True if we just learned that the Octopus will flash, meaning
        repeated calls will not let him flash again and will return False.
        """
        if 0 <= row < len(self.grid) and 0 <= col < len(self.grid[row]):
            return self.grid[row][col].increment_energy()

    def num_octopuses(self) -> int:
        """
        Returns a count of the grid size.
        """
        octopuses = 0
        for ridx, row in enumerate(self.grid):
            for cidx, col in enumerate(row):
                octopuses += 1
        return octopuses

    def reset_flashes(self) -> None:
        """
        Resets all Octopuses to the non-Flashed state.  A sort of internal cleanup
        between rounds.
        """
        for ridx, row in enumerate(self.grid):
            for cidx, col in enumerate(row):
                col.flashed_this_round = False

    def step(self) -> Tuple[int, bool]:
        """
        The all important step function that runs this logic.
        Returns 
            <int> count of the flashes this round
            <bool> if all Octopuses flashed this round.
        """

        step_flashes = 0
        flashes: list[Octopus] = []

        # we start by incrementing everyone by one.
        for ridx, row in enumerate(self.grid):
            for cidx, col in enumerate(row):
                flash_happened = self.increment_energy_level_at(ridx, cidx)
                if flash_happened:
                    flashes.append(self.grid[ridx][cidx])
                    step_flashes = step_flashes + 1

        # we increment the adjacent Octopuses to flashing Octopuses.
        # we continue this until there are no new flashes to account for.
        while flashes:
            new_flashes: list[Octopus] = []
            for octopus in flashes:
                for adj_row in range(octopus.row-1, octopus.row+2):
                    for adj_col in range(octopus.col-1, octopus.col+2):
                        if 0 <= adj_row < len(self.grid) and 0 <= adj_col < len(self.grid[adj_row]):
                            flash_happened = self.increment_energy_level_at(adj_row, adj_col)
                            if flash_happened:
                                new_flashes.append(self.grid[adj_row][adj_col])
                                step_flashes = step_flashes + 1

            # reset newly flashed octopuses because we've dealt with existing
            flashes = new_flashes

        all_flashed = False
        if step_flashes == self.num_octopuses():
            all_flashed = True

        # reset for the next go
        self.reset_flashes()

        return step_flashes, all_flashed


In [4]:
# Unit test the Octopus class.

# initial increase should not cause a flash.
o1 = Octopus(1,1, 8)
o1.increment_energy()
assert o1.engergy_level == 9
assert o1.flashed_this_round == False
print(o1)

# initial increment should cause a flash.
o2 = Octopus(1,1, 9)
o2.increment_energy()
assert o2.flashed_this_round == True

# additional energy boosts should not increment more or flash more.
o2.increment_energy()
o2.increment_energy()
assert o2.engergy_level == 0
assert o2.flashed_this_round == True
print(o2)

Octopus(row=1, col=1, engergy_level=9)
Octopus(row=1, col=1, engergy_level=0)


In [5]:
c = Cave(test_input_location())
flashes = 0
for _ in range(100):
    new_flashes, _ = c.step()
    flashes += new_flashes
assert flashes == 1656

In [6]:
c_live = Cave(input_location())
flashes = 0
for _ in range(100):
    new_flashes, _ = c_live.step()
    flashes += new_flashes
flashes

1588

### Part 2
At what step do they all flash at the same time?

In [7]:
c_live = Cave(input_location())
steps = 0
while True:
    steps += 1
    _, all = c_live.step()
    if all:
        break
steps

517